### Try-it 8.1: The "Best" Model

This module was all about regression and using Python's scikitlearn library to build regression models.  Below, a dataset related to real estate prices in California is given. While many of the assignments you have built and evaluated different models, it is important to spend some time interpreting the resulting "best" model.  


Your goal is to build a regression model to predict the price of a house in California.  After doing so, you are to *interpret* the model.  There are many strategies for doing so, including some built in methods from scikitlearn.  One example is `permutation_importance`.  Permutation feature importance is a strategy for inspecting a model and its features importance.  

Take a look at the user guide for `permutation_importance` [here](https://scikit-learn.org/stable/modules/permutation_importance.html).  Use  the `sklearn.inspection` modules implementation of `permutation_importance` to investigate the importance of different features to your regression models.  Share these results on the discussion board.

In [395]:
import pandas as pd
from sklearn.inspection import permutation_importance
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector, ColumnTransformer
from sklearn.metrics import mean_squared_error 

from sklearn.model_selection import train_test_split

from sklearn import set_config
set_config(display="diagram") 

In [301]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [302]:
cali = pd.read_csv('data/housing.csv')

In [303]:
cali.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [305]:
cali.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [383]:
cali.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [306]:
# Preprocessing for Columns with Missing Values.

# Display and Clean rows having columns with missing values

# Method 1: Using isna() and sum() functions
missing_values = cali.isna().sum()

# Method 2: Using isnull() and sum() functions (equivalent to isna())
#missing_values = cali.isnull().sum()

# Display columns with missing values
columns_with_missing_values = missing_values[missing_values > 0]
print(columns_with_missing_values)

# Get the list of columns with missing values
columns_to_check = columns_with_missing_values.index.tolist()

# Drop rows where any of the identified columns have missing values
cali_cleaned = cali.dropna(subset=columns_to_check)

# Check if there are any missing values left in the specified columns
print("\nMissing values in cleaned DataFrame:")
print(cali_cleaned.isna().sum())

# Display the cleaned DataFrame
print("\nDataFrame after dropping rows with missing values in specified columns:")
print(cali_cleaned.head())

total_bedrooms    207
dtype: int64

Missing values in cleaned DataFrame:
longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

DataFrame after dropping rows with missing values in specified columns:
   longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0    -122.23     37.88                41.0        880.0           129.0   
1    -122.22     37.86                21.0       7099.0          1106.0   
2    -122.24     37.85                52.0       1467.0           190.0   
3    -122.25     37.85                52.0       1274.0           235.0   
4    -122.25     37.85                52.0       1627.0           280.0   

   population  households  median_income  median_house_value ocean_proximity  
0       322.0       126.0         8.3252            452600.0        NEA

In [307]:
# Preprocessing - examining Category columns.
# Get the unique values of a specific column, for example, 'column_name'
unique_values = cali['ocean_proximity'].unique()

# Print the unique values
print(unique_values)

['NEAR BAY' '<1H OCEAN' 'INLAND' 'NEAR OCEAN' 'ISLAND']


In [309]:
# Identify X and y to be used to train the model with

# Since target variable to predict is 'median_house_value', we have X and y as below
X = cali.drop(columns='median_house_value')
y = cali.median_house_value


In [310]:
# Preprocessing - identify and treat variables having multi-collinearity by using VIF - variance inflation factor.

def vif(exogs, data):
    vif_dict = {}
    
    for exog in exogs: 
        not_exog = [i for i in exogs if i!= exog]
        
        X, y = data[not_exog], data[exog]
        
        r_squared = LinearRegression().fit(X,y).score(X,y)
        
        #calc the VIF
        vif = 1/(1-r_squared)
        
        vif_dict[exog] = vif
    
    return pd.DataFrame({"VIF": vif_dict})

# Call vif() after imputing missing values for variable 'total_bedrooms' 
imputer = SimpleImputer(strategy='median')
X['total_bedrooms'] = imputer.fit_transform(X[['total_bedrooms']])


X_numeric = X.select_dtypes(include=[np.number])
vif(X_numeric.columns, X_numeric).sort_values(ascending=False, by='VIF')

,VIF
households,28.284141
total_bedrooms,26.882085
total_rooms,12.126382
latitude,8.826607
longitude,8.703623
population,6.261680
median_income,1.688920
housing_median_age,1.259064


In [404]:
# Treat multi-collinearity in dataset and drop variable households and total_rooms with high VIF
X1 = X_numeric.drop(columns=['households', 'total_rooms'])

vif(X1.columns, X1).sort_values(ascending=False, by='VIF')

# Add back the category variable
X1['ocean_proximity'] = X['ocean_proximity']
X1.head()

,longitude,latitude,housing_median_age,total_bedrooms,population,median_income,ocean_proximity
0,-122.23,37.88,41.0,129.0,322.0,8.3252,NEAR BAY
1,-122.22,37.86,21.0,1106.0,2401.0,8.3014,NEAR BAY
2,-122.24,37.85,52.0,190.0,496.0,7.2574,NEAR BAY
3,-122.25,37.85,52.0,235.0,558.0,5.6431,NEAR BAY
4,-122.25,37.85,52.0,280.0,565.0,3.8462,NEAR BAY


In [405]:
# With DataSet preprocessed, do the Test and Train split (Train/Test = 70/30)
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y, test_size=0.3, random_state=22)




In [406]:
# Create a Pipeline to use while building the model. 
# Ordinal encode the category variable and impute variable with missing values.
poly_ordinal_impute_transformer = make_column_transformer(
    (OrdinalEncoder(categories=[['NEAR BAY', '<1H OCEAN', 'INLAND', 'NEAR OCEAN', 'ISLAND']]), ['ocean_proximity']),
    (PolynomialFeatures(), make_column_selector(dtype_include=np.number)),
    (StandardScaler(), make_column_selector(dtype_include=np.number)),  
    (SimpleImputer(strategy='median'), ['total_bedrooms']), remainder = 'passthrough') 


pipe = Pipeline([('transformer', poly_ordinal_impute_transformer), ('regressor', LinearRegression())])
pipe

Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ordinalencoder',
                                                  OrdinalEncoder(categories=[['NEAR '
                                                                              'BAY',
                                                                              '<1H '
                                                                              'OCEAN',
                                                                              'INLAND',
                                                                              'NEAR '
                                                                              'OCEAN',
                                                                              'ISLAND']]),
                                                  ['ocean_proximity']),
                                                 ('polynomialfeatures',
                                                  PolynomialFeatures(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x28bfd9510>),
                                                 ('standardscaler',
                                                  StandardScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x28c218690>),
                                                 ('simpleimputer',
                                                  SimpleImputer(strategy='median'),
                                                  ['total_bedrooms'])])),
                ('regressor', LinearRegression())])

In [407]:
# Build Linear Regression Model for degrees 1 through 5, and identify optimal model
train_mses = []
test_mses = []

predict_train_all = []  # List to store predict_train for all degrees
predict_test_all = []   # List to store predict_test for all degrees
i_values = range(1, 6)

for i in i_values:
    # Create pipeline with PolynomialFeatures degree i
    poly_ordinal_impute_transformer = make_column_transformer(
                            (OrdinalEncoder(categories=[['NEAR BAY', '<1H OCEAN', 'INLAND', 'NEAR OCEAN', 'ISLAND']]), ['ocean_proximity']),
                            (PolynomialFeatures(degree=i), make_column_selector(dtype_include=np.number)),
                            (StandardScaler(), make_column_selector(dtype_include=np.number)),  
                            (SimpleImputer(strategy='median'), ['total_bedrooms']), 
                            remainder='passthrough') 
                       
    pipe = Pipeline([('transformer', poly_ordinal_impute_transformer), ('regressor', LinearRegression())])

    # Fit on train
    pipe.fit(X1_train, y1_train)
 
    # Predict on train and test for i 
    predict_train = pipe.predict(X1_train)
    predict_test = pipe.predict(X1_test)
    
    # Save predictions for all degrees
    predict_train_all.append(predict_train)
    predict_test_all.append(predict_test)
    
    # Compute MSEs 
    train_mses.append(mean_squared_error(y1_train, predict_train))
    test_mses.append(mean_squared_error(y1_test, predict_test))

In [409]:
# Plot MSEs to find best/optimal model
MSE_experiment_results = pd.DataFrame({
    'k': i_values,
    'Train MSE': train_mses,
    'Test MSE': test_mses
})

# Create the plot
fig = px.line(MSE_experiment_results, x='k', y=MSE_experiment_results.columns[1:], markers=True)

# Update layout
fig.update_layout(
    font_size=20,
    xaxis_title='Polynomial Degree (k)',
    yaxis_title='MSE',
    legend_title='',
    legend=dict(
        x=0.07,
        y=0.94,
        bordercolor='Black',
        borderwidth=2
    ),
    margin=dict(l=50, r=50, b=0, t=1),
    showlegend=True
)

fig.show()

# Find best degree polynomial model and their corresponding MSEs
best_complexity = test_mses.index(min(test_mses)) + 1
best_mse = min(test_mses)


# Determine index of optimal degree - which is really best_complexity - 1
best_degree_index = np.argmin(test_mses)

# Print results
print(f'The best degree polynomial model is: {best_complexity}')
print(f'The smallest mean squared error on the test data is: {best_mse:.2f}')
print(f'The index of best degree model is: {best_degree_index}')

# For the best degree model, save predicted and actual median_house_values by adding to the train and test datasets
train_with_predictions = X1_train.copy()
train_with_predictions.loc[X1_train.index, 'predicted_median_house_value'] = predict_train_all[best_degree_index]
train_with_predictions.loc[X1_train.index, 'actual_median_house_value'] = y1_train

test_with_predictions = X1_test.copy()
test_with_predictions.loc[X1_test.index, 'predicted_median_house_value'] = predict_test_all[best_degree_index]
test_with_predictions.loc[X1_test.index, 'actual_median_house_value'] = y1_test

# Print predicted results on test data
print("Shape of predictions on test data :", predict_test_all[best_degree_index].shape)
print(f'Predicted median house values on test data : {predict_test_all[best_degree_index]}')

# Save datasets with predictions as CSV files
train_with_predictions.to_csv('data/train_with_predictions_for_best_model.csv', index=False)
test_with_predictions.to_csv('data/test_with_predictions_for_best_model.csv', index=False)


The best degree polynomial model is: 3
The smallest mean squared error on the test data is: 3969290239.71
The index of best degree model is: 2
Shape of predictions on test data : (6192,)
Predicted median house values on test data : [140774.04350084 266662.79594988  63401.03234714 ... 145630.43198937
 167408.91635364 162898.20991963]


In [359]:
import plotly.express as px

# Extract predicted and actual values for train data for best_degree_index
predicted_train = predict_train_all[best_degree_index]
actual_train = y1_train


# Extract predicted and real values for test data for best_degree_index
predicted_test = predict_test_all[best_degree_index]
actual_test = y1_test


# Create a DataFrame containing actual and predicted values for test data
test_data = pd.DataFrame({'Actual Median House Value': actual_test, 'Predicted Median House Value': predicted_test})

# Plot actual vs predicted values for test data
fig = px.scatter(test_data, x='Actual Median House Value', y='Predicted Median House Value', title='Test Data - Actual vs. Predicted Median House Values for degree 3 model')
fig.show()

In [398]:
# Calculate baseline model score
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y, test_size=0.3, random_state=22)

# Baseline Preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), make_column_selector(dtype_include=np.number)), 
        ('cat', OrdinalEncoder(categories=[['NEAR BAY', '<1H OCEAN', 'INLAND', 'NEAR OCEAN', 'ISLAND']]), ['ocean_proximity']),  
        ('total_bedrooms', SimpleImputer(strategy='median'), ['total_bedrooms'])  
    ],
    remainder='passthrough'
)

# Create a simple LinearRegression model
baseline_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])
# Train baseline model
baseline_model.fit(X1_train, y1_train)

# Evaluate baseline model. 
baseline_train_score = baseline_model.score(X1_train, y1_train)
baseline_test_score = baseline_model.score(X1_test, y1_test)

print("Training R^2 score with Baseline model:", baseline_train_score)
print("Test R^2 score with Baseline model:", baseline_test_score)


Training R^2 score with Baseline model: 0.6292599613542642
Test R^2 score with Baseline model: 0.6407365503098301


In [399]:
from sklearn.inspection import permutation_importance
import plotly.graph_objects as go

# Split the data
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y, test_size=0.3, random_state=22)

# With a function that can take degree as a parameter
def evaluate_model_with_permutation_importance(X1_train, X1_test, y1_train, y1_test, degree):
    
    # Create a column transformer with the specified degree
    poly_ordinal_impute_transformer = make_column_transformer(
        (OrdinalEncoder(categories=[['NEAR BAY', '<1H OCEAN', 'INLAND', 'NEAR OCEAN', 'ISLAND']]), ['ocean_proximity']),
        (PolynomialFeatures(degree=degree), make_column_selector(dtype_include=np.number)),
        (StandardScaler(), make_column_selector(dtype_include=np.number)),
        (SimpleImputer(strategy='median'), ['total_bedrooms']),
        remainder='passthrough'
    )

    # Create a pipeline with the column transformer and LinearRegression
    model = make_pipeline(poly_ordinal_impute_transformer, LinearRegression())

    # Train the model
    model.fit(X1_train, y1_train)

    # Evaluate the model
    train_score = model.score(X1_train, y1_train)
    test_score = model.score(X1_test, y1_test)

    print(f"Training R^2 score with degree {degree} model:", train_score)
    print(f"Test R^2 score with degree {degree} model:", test_score)

    # Calculate permutation importance
    result = permutation_importance(model, X1_test, y1_test, n_repeats=10, random_state=22)

    # Get feature importances
    importance = result.importances_mean

    # Print feature importances
    for feature, imp in zip(X1_train.columns, importance):
        print(f"Feature {feature}: Importance = {imp}")

    # Create a bar graph using Plotly
    figX = go.Figure(data=[go.Bar(x=X1_train.columns, y=importance, marker_color='skyblue')])

    # Update layout for better visualization
    figX.update_layout(title=f'Feature Importances for degree {degree} model',
                      xaxis=dict(title='Features'),
                      yaxis=dict(title='Importance'))

    # Return fig
    return figX

# Evaluate model with permutation importance for best degree model
fig = evaluate_model_with_permutation_importance(X1_train, X1_test, y1_train, y1_test, best_complexity)
fig.show()

# Evaluate model with permutation importance for degree 5 model
fig = evaluate_model_with_permutation_importance(X1_train, X1_test, y1_train, y1_test, 5)
fig.show()


Training R^2 score with degree 3 model: 0.7228312698858961
Test R^2 score with degree 3 model: 0.6885230431826338
Feature longitude: Importance = 2.4226551609317304
Feature latitude: Importance = 3.2346079024550534
Feature housing_median_age: Importance = 0.08299161544168861
Feature total_bedrooms: Importance = 1.419046012724737
Feature population: Importance = 1.129178859699262
Feature median_income: Importance = 0.8273510118685123
Feature ocean_proximity: Importance = 0.00398225589478225


Training R^2 score with degree 5 model: 0.7379391078800335
Test R^2 score with degree 5 model: -0.06560209255343108
Feature longitude: Importance = 3.304637719117866
Feature latitude: Importance = 5.420857258615869
Feature housing_median_age: Importance = 2.9464260001819715
Feature total_bedrooms: Importance = 295.59155975717425
Feature population: Importance = 658.1235050357736
Feature median_income: Importance = 1.632526924755118
Feature ocean_proximity: Importance = 3.0471700052103755e-09
